In [1]:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt

path = R'C:\Users\ringel\Lokal\Python-local\m290_cmos_calibr_0821\calibration'
fname = path + R'\Result of blitzv30-processed-final-rev.tif'

In [56]:
def unwarpProj(img, fname, imgpoints, scale=1):
    # shape (px)
    h, w = img.shape[:2]
    
    # corners
    cimg = pd.DataFrame(imgpoints[0][:,0,:])
    cimg.columns = ['ximg', 'yimg']
    cimg = cimg*scale

    # edge points for unwarp
    num_x = 43
    leup = 0
    lelo = cimg.shape[0] - num_x
    riup = num_x - 1
    rilo = -1
    src = np.float32([cimg.iloc[leup].values,
                    (cimg.iloc[lelo].values),
                    (cimg.iloc[riup].values),
                    (cimg.iloc[rilo].values)])

    lens = np.zeros(4)
    lens[0] = cimg.iloc[riup].values[0]-cimg.iloc[leup].values[0]
    lens[1] = cimg.iloc[riup].values[0]-cimg.iloc[leup].values[0]
    lens[2] = cimg.iloc[lelo].values[1]-cimg.iloc[leup].values[1]
    lens[3] = cimg.iloc[rilo].values[1]-cimg.iloc[riup].values[1]
    print(f"lengths of upper edge: {lens[0]}; lower: {lens[1]}; left: {lens[2]}; right: {lens[3]}")

    # unwarped must be square. dimensions should not be greater then longest edge of found quadrangle. more precise would be use hypotenuse above
    x = lens.max()
    y = x
    # offsets to show regions above and left besides in output image
    offsx = 100*scale
    offsy = 500*scale
    dst = np.float32([(0+offsx, 0+offsy),
                    (x+offsx, 0+offsy),
                    (0+offsx, y+offsy),
                    (x+offsx, y+offsy)])
    
    # M: transform matrix, Minv: the inverse
    M = cv2.getPerspectiveTransform(src, dst)
    # use cv2.warpPerspective() to warp your image to a top-down view
    warped = cv2.warpPerspective(img, M, (h, w), flags=cv2.INTER_LINEAR)
    rotated = cv2.rotate(warped, cv2.cv2.ROTATE_90_COUNTERCLOCKWISE)
    flipped = cv2.flip(rotated, 0)
    fout = fname.rsplit('.', 1)[0] + '_unwarp.tif'
    cv2.imwrite(fout, flipped)
    return flipped, fout, M


def chessCorners(img, fname, x=43, y=43):
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((x*y,3), np.float32)
    objp[:,:2] = np.mgrid[0:x,0:y].T.reshape(-1,2)
    # Arrays to store object points and image points from all images
    objpoints = [] # 3d point in real world space
    imgpoints = [] # 2d points in image plane.

    #chessboard_flags = cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FILTER_QUADS + cv2.CALIB_CB_NORMALIZE_IMAGE
    chessboard_flags = 0

    if img.ndim == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    if img.ndim == 2:
        gray = img.copy()
    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (x,y), chessboard_flags)
    # If found, add object points, image points (after refining them)
    n = corners.shape[0]
    file = fname.rsplit('\\', 1)[-1]
    if ret == True:
        print(f'{n} corners found in {file}')
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        df_corners = pd.DataFrame(corners2[:,0])
        df_corners.columns = ['x', 'y']
        pos = fname.rfind('.')
        f = fname[:pos] + '_corners.h5'
        df_corners.to_hdf(f, key='df', mode='w')
        imgpoints.append(corners)
        # Draw and display the corners
        # cv2.drawChessboardCorners(img, (x,y), corners2, ret)
        cv2.drawChessboardCorners(img, (x,y), corners2, ret)
        f = fname[:pos] + '_corners.tif'
        cv2.imwrite(f, img)
        return gray, objpoints, imgpoints
    else:
        print(f'No cornes could be found. Check preprocessing of input image.')

def undistort(img, fname, objpoints, imgpoints, scale=1):
    #objpoints = objpoints*scale
    if scale != 1:
        imgpoints2[0] = imgpoints2[0]*scale
    if img.ndim == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    retval, cameraMatrix, distCoeffs, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[::-1], None, None)
    # images shape
    h, w = img.shape[:2]
    newCamMatrix, roi = cv2.getOptimalNewCameraMatrix(cameraMatrix, distCoeffs, (w,h), 1, (w,h))

    undist = cv2.undistort(img, cameraMatrix, distCoeffs, None, newCamMatrix)
    x, y, w, h = roi
    undist = undist[y:y+h, x:x+w]
    fout = fname.rsplit('.', 1)[0] + '_undist.tif'
    cv2.imwrite(fout, undist)
    
    return undist, fout, cameraMatrix, distCoeffs



Workflow:

In [66]:
# set binarized correction image (used function findChessCorners is not able to process large images because filtersize is not changeable)
# alternatively image could be devided into 4 subimages (quadrants) for cornerpointsearch and stitched together again
# altough calibration could be done with themal signature and mimimum filter to avoid issues due to reflections and lightning
fname = path + R'\blitzv30-fullscale----final-manuallyfill-medianR10.tif'
# import image with opencv
img = cv2.imread(fname)
# find cornerpoints
img, objpoints, imgpoints = chessCorners(img, fname)
# unwarp without scaling. file will be saved automatically
img, fname, M = unwarpProj(img, fname, imgpoints, scale=1)
# find corners again in unwarped image
img, _, imgpoints2 = chessCorners(img, fname)
# undistort unwarped image
img, fname, cameraMatrix, distCoeffs = undistort(img, fname, objpoints, imgpoints2)
# find corners again
img, _, imgpoints3 = chessCorners(img, fname)

No cornes could be found. Check preprocessing of input image.


TypeError: cannot unpack non-iterable NoneType object

Mit der Liste aus punkten in der letzten corners.h5 muss jetzt die skalierte maschinenkos korelation passieren

In [59]:
np.array_equal(objpoints,objpoints2)

True

Apply to another image

In [57]:
# image to process
forig = R'C:\Users\ringel\Lokal\Python-local\m290_cmos_calibr_0821\cmoslayers\chessboard_eos_10s_f8_ext-flash30'
# read with opencv
imgorig = cv2.imread(forig)
# unwarp with scaling regarding scaling of correction image 
imgorig, fname, M = unwarpProj(imgorig, forig, imgpoints, scale=4)
# undistort with scaling regarding scaling of correction image 
imgorig, fname, cameraMatrix, distCoeffs = undistort(imgorig, fname, objpoints, imgpoints2, scale=4)

lengths of upper edge: 5334.9140625; lower: 5334.9140625; left: 5261.421875; right: 5616.5068359375


In [4]:
layer = pd.read_hdf(R'C:\Users\ringel\Lokal\Python-local\ProSi3D\brosev2_19112021\res\eosxytime_00010.h5')

In [7]:
layer.loc[layer.time > 0].head()

,time,lse,ttl,kse_bp,kse_rec,x_mm,y_mm,exposureType,partId,weldtime,weldId
176110,0.000042,0.005634,4.532682,0.096516,-0.022952,72.163429,3.08,10.0,5.0,NaN,0.0
176111,0.000083,0.004596,4.638347,0.096612,-0.003456,72.246857,3.08,10.0,5.0,NaN,0.0
176112,0.000125,0.003798,4.677065,0.095507,-0.012271,72.330284,3.08,10.0,5.0,NaN,0.0
176113,0.000167,0.003474,4.695788,0.093767,-0.001354,72.413719,3.08,10.0,5.0,NaN,0.0
176114,0.000208,0.003050,4.712968,0.092993,0.001016,72.497147,3.08,10.0,5.0,NaN,0.0
